In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import cm as colormaps
from scipy import interpolate

from importlib import reload

import environment as Env
import ray as Ray
import simulation as Sim

from topography import bathymetry as Bathy

In [2]:
reload(Bathy)

map_path = 'src/asc/MNT_MED100m_GDL-CA_HOMONIM_WGS84_NM_ZNEG.asc'
map = Bathy.Map.from_asc(map_path)

In [3]:
start = (2.8995, 41.6995)  # lower left
stop = (7.9005, 41.6995)  # lower right

x, z = map.cut(start, stop, npoints=100)
floor = interpolate.interp1d(x, z)

ceiling = lambda x: 0 * np.ones_like(x)

In [45]:
reload(Ray)
reload(Env)
reload(Sim)

source = np.array([0., 0.])
target = np.array([3500, -1000])

env = Env.Environment2D (floor, ceiling)

sim = Sim.EigenraySim2D (
    env, source, target,
    dx_max=50, dz_max=25, backprop=False, n_rebounds_max=6,
    n_rays_scan=1000, n_rays=25, scan_angle_min=-1.5, scan_angle_max=1.5,
    verbose=True, verbose_depth_max=0
)

sim.refine(n_rays=20, cone_half_angle=0.01)  # Initial refine (empirical: cone_half_angle = 10/n_rays_scan)
sim.refine(iterations=5)

EigenraySim2D: Scanning using 1000 rays between angles -1.5 and 1.5
EigenraySim2D: Scan mean distance: 7.048640431018566
EigenraySim2D: Refine #1 mean distance: 0.9467889555897727
EigenraySim2D: Refine #2 mean distance: 0.43452913625601924
EigenraySim2D: Refine #3 mean distance: 0.1887311825188499
EigenraySim2D: Refine #4 mean distance: 0.0580425925032614
EigenraySim2D: Target precision reached


In [58]:
from scipy.io import wavfile

sample_rate, signal = wavfile.read('src/wav/sig-test.wav')
signal = signal[:, 0]

In [59]:
freqs = np.linspace(0, 24000, 100)
filter = sim.get_filter (*freqs)

sample_rate, signal_filtered = filter(sample_rate, signal)
signal_filtered = signal_filtered.astype(signal.dtype)
wavfile.write('src/wav/sig-test-filtered.wav', sample_rate, signal_filtered.astype(np.int16))